In [43]:
# # #5/19 테스트
# from google.colab import drive
# drive.mount('/content/drive')
import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks/swishnet')
# import sys
# sys.path.append(os.getcwd())
# !pip install import-ipynb

In [ ]:
cutting_size = 20000 # 16000~48000
path = "./test/car_2.wav"

In [45]:
import torch
import torchaudio
import import_ipynb

from model import swishnet

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [47]:
waveform, sample_rate = torchaudio.load(path)
print("원래 shape:", waveform.shape)

# 다채널이면 평균내서 모노로
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Resample
resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
resampled_waveform = resampler(waveform)

# 1채널이므로 squeeze로 1D 텐서 만들기
resampled_waveform = resampled_waveform.squeeze(0)
print(resampled_waveform.shape)
print("최종 shape:", resampled_waveform.shape)  # torch.Size([샘플 수])
print("샘플 수:", resampled_waveform.shape[0])
print("길이 (초):", resampled_waveform.shape[0] / 16000)

원래 shape: torch.Size([2, 220500])
torch.Size([80000])
최종 shape: torch.Size([80000])
샘플 수: 80000
길이 (초): 5.0


In [48]:
model = swishnet(sample_rate = 16000).to(device)
model.load_state_dict(torch.load('./swishnet_times_0_epoch_410(random).pth'))
model.eval()

swishnet(
  (dropout): Dropout(p=0.2, inplace=False)
  (mfcc_transform): MFCC(
    (amplitude_to_DB): AmplitudeToDB()
    (MelSpectrogram): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (block_1_up): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(20, 16, kernel_size=(3,), stride=(1,))
    )
  )
  (block_1_down): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(20, 16, kernel_size=(6,), stride=(1,))
    )
  )
  (block_2_up): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(32, 8, kernel_size=(3,), stride=(1,))
    )
  )
  (block_2_down): swishnet_module(
    (tanh): Tanh()
    (sig): Sigmoid()
    (CausalConv1d): CausalConv1d(
      (conv): Conv1d(32, 8, kernel_size=(6,), stride=(1,))
    )
  )
  (block_3_up): swishnet_module(
    (tanh): Tanh()
    (si

In [49]:
num_segments = resampled_waveform.shape[0] // cutting_size

for i in range(num_segments):
    with torch.no_grad():
        cutted = resampled_waveform[i * cutting_size : (i + 1) * cutting_size]
        cutted = cutted.unsqueeze(0).unsqueeze(0) # (1, 16000) 모양으로 배치 차원 추가
        pred = model(cutted.to(device))   # 모델도 GPU에 올라가 있어야 함
        probs = torch.softmax(pred, dim=1)
    #car 0
    #music= 1
    #speech = 2
    #noise = 3
        preds = probs.argmax(dim=1)# 🔸예측 클래스 index
        print(preds)

tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
